In [1]:
# Import all packages required for this project. 
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import norm
import sys
import pandas
from sklearn.multiclass import OneVsRestClassifier
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [2]:
# Read the files and name them accordingly
iplmatches = pd.read_csv('Matches.csv')
ipldelivery = pd.read_csv('Deliveries.csv')

In [3]:
#Ensure that the file is being read correctly
ipldelivery.head(5)
# iplmatches.head(3)


,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [4]:

batsman_grp = ipldelivery.groupby(["match_id", "inning", "batting_team", "batsman"])
batsman_grp.head(5)
batsman_grp["batsman_runs"].sum().reset_index()


,match_id,inning,batting_team,batsman,batsman_runs
0,1,1,Sunrisers Hyderabad,BCJ Cutting,16
1,1,1,Sunrisers Hyderabad,DA Warner,14
2,1,1,Sunrisers Hyderabad,DJ Hooda,16
3,1,1,Sunrisers Hyderabad,MC Henriques,52
4,1,1,Sunrisers Hyderabad,S Dhawan,40
...,...,...,...,...,...
11330,11415,2,Chennai Super Kings,MS Dhoni,2
11331,11415,2,Chennai Super Kings,RA Jadeja,9
11332,11415,2,Chennai Super Kings,SK Raina,9
11333,11415,2,Chennai Super Kings,SN Thakur,2


In [5]:
## BATSMEN DATA GROUPED BY MATCH
# Here the data is grouped to provide deeper depth of statistics and later for the team classificaiton

batsman_grp = ipldelivery.groupby(["match_id", "inning", "batting_team", "batsman"])
batsman_grp.head(5)
batsmen = batsman_grp["batsman_runs"].sum().reset_index()
balls_faced = ipldelivery[ipldelivery["wide_runs"] == 0]
balls_faced = balls_faced.groupby(["match_id", "inning", "batsman"])["batsman_runs"].count().reset_index()
balls_faced.columns = ["match_id", "inning", "batsman", "balls_faced"]
batsmen = batsmen.merge(balls_faced, left_on=["match_id", "inning", "batsman"], right_on=["match_id", "inning", "batsman"], how="left")
batsmen

fours = ipldelivery[ ipldelivery["batsman_runs"] == 4]
sixes = ipldelivery[ ipldelivery["batsman_runs"] == 6]

sixes
fours_per_batsman = fours.groupby(["match_id", "inning", "batsman"])["batsman_runs"].count().reset_index()
sixes_per_batsman = sixes.groupby(["match_id", "inning", "batsman"])["batsman_runs"].count().reset_index()

fours_per_batsman.columns = ["match_id", "inning", "batsman", "4s"]
sixes_per_batsman.columns = ["match_id", "inning", "batsman", "6s"]
batsmen = batsmen.merge(fours_per_batsman, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")
batsmen = batsmen.merge(sixes_per_batsman, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")
batsmen['SR'] = np.round(batsmen['batsman_runs'] / batsmen['balls_faced'] * 100, 2)
batsmen

for col in ["batsman_runs", "4s", "6s", "balls_faced", "SR"]:
    batsmen[col] = batsmen[col].fillna(0)

dismissals = ipldelivery[ pd.notnull(ipldelivery["player_dismissed"])]
dismissals = dismissals[["match_id", "inning", "player_dismissed", "dismissal_kind", "fielder"]]
dismissals.rename(columns={"player_dismissed": "batsman"}, inplace=True)
dismissals

batsmen = batsmen.merge(dismissals, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")

batsmen = iplmatches[['id','season']].merge(batsmen, left_on = 'id', right_on = 'match_id', how = 'left').drop('id', axis = 1)
batsmen.head(10)

,season,match_id,inning,batting_team,batsman,batsman_runs,balls_faced,4s,6s,SR,dismissal_kind,fielder
0,2008,60,1,Kolkata Knight Riders,BB McCullum,158,73.0,10.0,13.0,216.44,NaN,NaN
1,2008,60,1,Kolkata Knight Riders,DJ Hussey,12,12.0,1.0,0.0,100.00,caught,CL White
2,2008,60,1,Kolkata Knight Riders,Mohammad Hafeez,5,3.0,1.0,0.0,166.67,NaN,NaN
3,2008,60,1,Kolkata Knight Riders,RT Ponting,20,20.0,1.0,1.0,100.00,caught,P Kumar
4,2008,60,1,Kolkata Knight Riders,SC Ganguly,10,12.0,2.0,0.0,83.33,caught,JH Kallis
5,2008,60,2,Royal Challengers Bangalore,AA Noffke,9,10.0,1.0,0.0,90.00,run out,AB Agarkar
6,2008,60,2,Royal Challengers Bangalore,B Akhil,0,2.0,0.0,0.0,0.00,caught,RT Ponting
7,2008,60,2,Royal Challengers Bangalore,CL White,6,10.0,0.0,0.0,60.00,caught,WP Saha
8,2008,60,2,Royal Challengers Bangalore,JH Kallis,8,7.0,0.0,1.0,114.29,caught,M Kartik
9,2008,60,2,Royal Challengers Bangalore,MV Boucher,7,9.0,1.0,0.0,77.78,caught,M Kartik


In [6]:
cpy_batsmen = batsmen
cpy_batsmen =cpy_batsmen.drop(columns = ['batsman','SR', 'balls_faced', '4s', '6s', 'dismissal_kind', 'fielder'])
cpy_batsmen = cpy_batsmen.groupby(["season","match_id", "inning", "batting_team"])
cpy_batsmen.head()

team_runs = cpy_batsmen["batsman_runs"].sum().reset_index()
team_runs=team_runs.rename(columns={'batsman_runs': "avgRuns"})

In [7]:
team_runs

,season,match_id,inning,batting_team,avgRuns
0,2008,60,1,Kolkata Knight Riders,205
1,2008,60,2,Royal Challengers Bangalore,63
2,2008,61,1,Chennai Super Kings,234
3,2008,61,2,Kings XI Punjab,196
4,2008,62,1,Rajasthan Royals,122
...,...,...,...,...,...
1523,2019,11413,2,Delhi Capitals,165
1524,2019,11414,1,Delhi Capitals,147
1525,2019,11414,2,Chennai Super Kings,151
1526,2019,11415,1,Mumbai Indians,149


In [8]:
runs_year = team_runs.groupby(['season', 'batting_team']).mean()
runs_year= runs_year.drop(columns= ['match_id', 'inning'])
dict1 = dict()
for i,j in team_runs.groupby(['season', 'batting_team']):
    print(i)
    print(type(i))
    print(i[0])
    dict1[i] = j['avgRuns'].mean()
    

(2008, 'Chennai Super Kings')
<class 'tuple'>
2008
(2008, 'Deccan Chargers')
<class 'tuple'>
2008
(2008, 'Delhi Daredevils')
<class 'tuple'>
2008
(2008, 'Kings XI Punjab')
<class 'tuple'>
2008
(2008, 'Kolkata Knight Riders')
<class 'tuple'>
2008
(2008, 'Mumbai Indians')
<class 'tuple'>
2008
(2008, 'Rajasthan Royals')
<class 'tuple'>
2008
(2008, 'Royal Challengers Bangalore')
<class 'tuple'>
2008
(2009, 'Chennai Super Kings')
<class 'tuple'>
2009
(2009, 'Deccan Chargers')
<class 'tuple'>
2009
(2009, 'Delhi Daredevils')
<class 'tuple'>
2009
(2009, 'Kings XI Punjab')
<class 'tuple'>
2009
(2009, 'Kolkata Knight Riders')
<class 'tuple'>
2009
(2009, 'Mumbai Indians')
<class 'tuple'>
2009
(2009, 'Rajasthan Royals')
<class 'tuple'>
2009
(2009, 'Royal Challengers Bangalore')
<class 'tuple'>
2009
(2010, 'Chennai Super Kings')
<class 'tuple'>
2010
(2010, 'Deccan Chargers')
<class 'tuple'>
2010
(2010, 'Delhi Daredevils')
<class 'tuple'>
2010
(2010, 'Kings XI Punjab')
<class 'tuple'>
2010
(2010, 'K

In [10]:
# team_runs[team_runs['season'] == 2008 and team_runs['inning'] == 1]
filter1  = team_runs['season'] ==2008
filter2 = team_runs['batting_team'] == 'Kolkata Knight Riders'

a = team_runs.where(filter1&filter2)
a=a.dropna().reset_index()
a= a.drop(columns = 'index') 
a['avgRuns'] = 200

# for i in a.iterrows():

# for i, row  in a.iterrows():
#     print(row)
#     if row[3] == 'Kolkata Knight Riders':
#         a.at[i, 'avgRuns'] = 201

for i, ro in a.iterrows():
      for j, row in team_runs.iterrows():
            if(ro[0] == row[0] and ro[3] == row[3]):
                team_runs.at[j , 'avgRuns']=ro[4] 
                

a

,season,match_id,inning,batting_team,avgRuns
0,2008.0,60.0,1.0,Kolkata Knight Riders,200
1,2008.0,64.0,2.0,Kolkata Knight Riders,200
2,2008.0,71.0,1.0,Kolkata Knight Riders,200
3,2008.0,75.0,1.0,Kolkata Knight Riders,200
4,2008.0,78.0,2.0,Kolkata Knight Riders,200
5,2008.0,81.0,2.0,Kolkata Knight Riders,200
6,2008.0,88.0,1.0,Kolkata Knight Riders,200
7,2008.0,92.0,1.0,Kolkata Knight Riders,200
8,2008.0,95.0,1.0,Kolkata Knight Riders,200
9,2008.0,99.0,1.0,Kolkata Knight Riders,200


In [116]:
team_runs.where(filter1 & filter2).dropna()

,season,match_id,inning,batting_team,avgRuns
0,2008.0,60.0,1.0,Kolkata Knight Riders,200.0
9,2008.0,64.0,2.0,Kolkata Knight Riders,200.0
22,2008.0,71.0,1.0,Kolkata Knight Riders,200.0
30,2008.0,75.0,1.0,Kolkata Knight Riders,200.0
37,2008.0,78.0,2.0,Kolkata Knight Riders,200.0
43,2008.0,81.0,2.0,Kolkata Knight Riders,200.0
56,2008.0,88.0,1.0,Kolkata Knight Riders,200.0
64,2008.0,92.0,1.0,Kolkata Knight Riders,200.0
70,2008.0,95.0,1.0,Kolkata Knight Riders,200.0
78,2008.0,99.0,1.0,Kolkata Knight Riders,200.0


In [124]:
for i , j in dict1:
    filter1 = team_runs['season'] == i
    filter2 = team_runs['batting_team'] == j
    a = team_runs.where(filter1&filter2)
    a=a.dropna().reset_index()
    a= a.drop(columns = 'index') 
    if i = 2008 :
        a['avgRuns'] = 0
    else:
        a['avgRuns'] = dict1[i-1,j]
    
    for k, ro in a.iterrows():
          for l, row in team_runs.iterrows():
                if(ro[0] == row[0] and ro[3] == row[3]):
                    team_runs.at[l , 'avgRuns']=ro[4] 


    

In [126]:
team_runs.to_csv('AvgRunsPerSeason.csv')

In [130]:
a['avgRuns'] = dict1[2010,'Chennai Super Kings']

In [128]:
dict1

{(2008, 'Chennai Super Kings'): 147.8125,
 (2008, 'Deccan Chargers'): 151.21428571428572,
 (2008, 'Delhi Daredevils'): 142.85714285714286,
 (2008, 'Kings XI Punjab'): 156.0,
 (2008, 'Kolkata Knight Riders'): 138.53846153846155,
 (2008, 'Mumbai Indians'): 135.5,
 (2008, 'Rajasthan Royals'): 151.5,
 (2008, 'Royal Challengers Bangalore'): 133.21428571428572,
 (2009, 'Chennai Super Kings'): 148.85714285714286,
 (2009, 'Deccan Chargers'): 142.6875,
 (2009, 'Delhi Daredevils'): 134.73333333333332,
 (2009, 'Kings XI Punjab'): 126.28571428571429,
 (2009, 'Kolkata Knight Riders'): 119.85714285714286,
 (2009, 'Mumbai Indians'): 135.84615384615384,
 (2009, 'Rajasthan Royals'): 115.0,
 (2009, 'Royal Challengers Bangalore'): 135.375,
 (2010, 'Chennai Super Kings'): 143.64705882352942,
 (2010, 'Deccan Chargers'): 140.25,
 (2010, 'Delhi Daredevils'): 144.85714285714286,
 (2010, 'Kings XI Punjab'): 144.33333333333334,
 (2010, 'Kolkata Knight Riders'): 141.28571428571428,
 (2010, 'Mumbai Indians'): 159

In [133]:
# same thing to get avg wckts

## Bowlers grouped by sets of data
# Data is grouped for bowlers to provide greater depth of information. Very important for the regression analysis.

bowler_grp = ipldelivery.groupby(["match_id", "inning", "bowling_team", "bowler", "over"])
bowlers = bowler_grp["total_runs", "wide_runs", "bye_runs", "legbye_runs", "noball_runs"].sum().reset_index()

bowlers["runs"] = bowlers["total_runs"] - (bowlers["bye_runs"] + bowlers["legbye_runs"])
bowlers["extras"] = bowlers["wide_runs"] + bowlers["noball_runs"]

del( bowlers["bye_runs"])
del( bowlers["legbye_runs"])
del( bowlers["total_runs"])

dismissal_kinds_for_bowler = ["bowled", "caught", "lbw", "stumped", "caught and bowled", "hit wicket"]
dismissals = ipldelivery[ipldelivery["dismissal_kind"].isin(dismissal_kinds_for_bowler)]
dismissals = dismissals.groupby(["match_id", "inning", "bowling_team", "bowler", "over"])["dismissal_kind"].count().reset_index()
dismissals.rename(columns={"dismissal_kind": "wickets"}, inplace=True)

bowlers = bowlers.merge(dismissals, left_on=["match_id", "inning", "bowling_team", "bowler", "over"], 
                        right_on=["match_id", "inning", "bowling_team", "bowler", "over"], how="left")
bowlers["wickets"] = bowlers["wickets"].fillna(0)

bowlers_over = bowlers.groupby(['match_id', 'inning', 'bowling_team', 'bowler'])['over'].count().reset_index()
bowlers = bowlers.groupby(['match_id', 'inning', 'bowling_team', 'bowler']).sum().reset_index().drop('over', 1)
bowlers = bowlers_over.merge(bowlers, on=["match_id", "inning", "bowling_team", "bowler"], how = 'left')
bowlers['Econ'] = np.round(bowlers['runs'] / bowlers['over'] , 2)
bowlers = iplmatches[['id','season']].merge(bowlers, left_on = 'id', right_on = 'match_id', how = 'left').drop('id', axis = 1)

bowlers.head(10)

<ipython-input-133-b057376b7608>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bowlers = bowler_grp["total_runs", "wide_runs", "bye_runs", "legbye_runs", "noball_runs"].sum().reset_index()


,season,match_id,inning,bowling_team,bowler,over,wide_runs,noball_runs,runs,extras,wickets,Econ
0,2008,60,1,Royal Challengers Bangalore,AA Noffke,4,5,0,40,5,1.0,10.00
1,2008,60,1,Royal Challengers Bangalore,CL White,1,2,0,24,2,0.0,24.00
2,2008,60,1,Royal Challengers Bangalore,JH Kallis,4,1,0,48,1,1.0,12.00
3,2008,60,1,Royal Challengers Bangalore,P Kumar,4,1,0,38,1,0.0,9.50
4,2008,60,1,Royal Challengers Bangalore,SB Joshi,3,0,0,26,0,0.0,8.67
5,2008,60,1,Royal Challengers Bangalore,Z Khan,4,0,0,38,0,1.0,9.50
6,2008,60,2,Kolkata Knight Riders,AB Agarkar,4,4,0,25,4,3.0,6.25
7,2008,60,2,Kolkata Knight Riders,AB Dinda,3,2,0,9,2,2.0,3.00
8,2008,60,2,Kolkata Knight Riders,I Sharma,3,1,0,7,1,1.0,2.33
9,2008,60,2,Kolkata Knight Riders,LR Shukla,2,3,0,12,3,1.0,6.00


In [145]:
cpy_bowler = bowlers
cpy_bowler =cpy_bowler.drop(columns = ['bowler','over', 'wide_runs', 'noball_runs', 'runs', 'extras', 'Econ'])
cpy_bowler = cpy_bowler.groupby(["season","match_id", "bowling_team"])
cpy_bowler.head()

team_wickets = cpy_bowler["wickets"].sum().reset_index()
team_wickets=team_wickets.rename(columns={'wickets': "avgWickets"})
team_wickets

,season,match_id,bowling_team,avgWickets
0,2008,60,Kolkata Knight Riders,9.0
1,2008,60,Royal Challengers Bangalore,3.0
2,2008,61,Chennai Super Kings,4.0
3,2008,61,Kings XI Punjab,5.0
4,2008,62,Delhi Daredevils,6.0
...,...,...,...,...
1505,2019,11413,Sunrisers Hyderabad,6.0
1506,2019,11414,Chennai Super Kings,9.0
1507,2019,11414,Delhi Capitals,4.0
1508,2019,11415,Chennai Super Kings,7.0


In [148]:
wickets_year = team_wickets.groupby(['season', 'bowling_team']).mean()
wickets_year= wickets_year.drop(columns= ['match_id'])
dict2 = dict()
for i,j in team_wickets.groupby(['season', 'bowling_team']):
    print(i)
    print(type(i))
    print(i[0])
    dict2[i] = j['avgWickets'].mean()
    

(2008, 'Chennai Super Kings')
<class 'tuple'>
2008
(2008, 'Deccan Chargers')
<class 'tuple'>
2008
(2008, 'Delhi Daredevils')
<class 'tuple'>
2008
(2008, 'Kings XI Punjab')
<class 'tuple'>
2008
(2008, 'Kolkata Knight Riders')
<class 'tuple'>
2008
(2008, 'Mumbai Indians')
<class 'tuple'>
2008
(2008, 'Rajasthan Royals')
<class 'tuple'>
2008
(2008, 'Royal Challengers Bangalore')
<class 'tuple'>
2008
(2009, 'Chennai Super Kings')
<class 'tuple'>
2009
(2009, 'Deccan Chargers')
<class 'tuple'>
2009
(2009, 'Delhi Daredevils')
<class 'tuple'>
2009
(2009, 'Kings XI Punjab')
<class 'tuple'>
2009
(2009, 'Kolkata Knight Riders')
<class 'tuple'>
2009
(2009, 'Mumbai Indians')
<class 'tuple'>
2009
(2009, 'Rajasthan Royals')
<class 'tuple'>
2009
(2009, 'Royal Challengers Bangalore')
<class 'tuple'>
2009
(2010, 'Chennai Super Kings')
<class 'tuple'>
2010
(2010, 'Deccan Chargers')
<class 'tuple'>
2010
(2010, 'Delhi Daredevils')
<class 'tuple'>
2010
(2010, 'Kings XI Punjab')
<class 'tuple'>
2010
(2010, 'K

In [149]:
dict2

{(2008, 'Chennai Super Kings'): 5.1875,
 (2008, 'Deccan Chargers'): 4.285714285714286,
 (2008, 'Delhi Daredevils'): 5.857142857142857,
 (2008, 'Kings XI Punjab'): 5.533333333333333,
 (2008, 'Kolkata Knight Riders'): 4.6923076923076925,
 (2008, 'Mumbai Indians'): 5.928571428571429,
 (2008, 'Rajasthan Royals'): 6.0,
 (2008, 'Royal Challengers Bangalore'): 4.0,
 (2009, 'Chennai Super Kings'): 6.0,
 (2009, 'Deccan Chargers'): 6.0625,
 (2009, 'Delhi Daredevils'): 6.066666666666666,
 (2009, 'Kings XI Punjab'): 5.071428571428571,
 (2009, 'Kolkata Knight Riders'): 3.769230769230769,
 (2009, 'Mumbai Indians'): 5.538461538461538,
 (2009, 'Rajasthan Royals'): 6.076923076923077,
 (2009, 'Royal Challengers Bangalore'): 4.9375,
 (2010, 'Chennai Super Kings'): 5.625,
 (2010, 'Deccan Chargers'): 5.8125,
 (2010, 'Delhi Daredevils'): 5.785714285714286,
 (2010, 'Kings XI Punjab'): 4.642857142857143,
 (2010, 'Kolkata Knight Riders'): 4.857142857142857,
 (2010, 'Mumbai Indians'): 5.375,
 (2010, 'Rajasthan 

In [168]:
filter1  = team_wickets['season'] ==2008
filter2 = team_wickets['bowling_team'] == 'Kolkata Knight Riders'

a = team_wickets.where(filter1&filter2)
a=a.dropna().reset_index()
a= a.drop(columns = 'index') 
a['avgWickets'] = 6

for k, ro in a.iterrows():
    for l, row in team_wickets.iterrows():
        if(ro[0] == row[0] and ro[2] == row[2]):
            team_wickets.at[l , 'avgWickets']=ro[3] 
    break
team_wickets.where(filter1 & filter2).dropna()

,season,match_id,bowling_team,avgWickets
0,2008.0,60.0,Kolkata Knight Riders,6.0
9,2008.0,64.0,Kolkata Knight Riders,6.0
23,2008.0,71.0,Kolkata Knight Riders,6.0
30,2008.0,75.0,Kolkata Knight Riders,6.0
36,2008.0,78.0,Kolkata Knight Riders,6.0
43,2008.0,81.0,Kolkata Knight Riders,6.0
56,2008.0,88.0,Kolkata Knight Riders,6.0
65,2008.0,92.0,Kolkata Knight Riders,6.0
71,2008.0,95.0,Kolkata Knight Riders,6.0
78,2008.0,99.0,Kolkata Knight Riders,6.0


In [174]:
for i , j in dict2:
    filter1 = team_wickets['season'] == i
    filter2 = team_wickets['bowling_team'] == j
    a = team_wickets.where(filter1&filter2)
    a=a.dropna().reset_index()
    a= a.drop(columns = 'index') 
    a['avgWickets'] = dict2[i,j]
    print(a)

    for k, ro in a.iterrows():
        for l, row in team_wickets.iterrows():
            if(ro[0] == row[0] and ro[2] == row[2]):
                team_wickets.at[l , 'avgWickets']=int(ro[3]) 
        break



    season  match_id         bowling_team  avgWickets
0   2008.0      61.0  Chennai Super Kings      5.1875
1   2008.0      67.0  Chennai Super Kings      5.1875
2   2008.0      71.0  Chennai Super Kings      5.1875
3   2008.0      74.0  Chennai Super Kings      5.1875
4   2008.0      79.0  Chennai Super Kings      5.1875
5   2008.0      83.0  Chennai Super Kings      5.1875
6   2008.0      85.0  Chennai Super Kings      5.1875
7   2008.0      87.0  Chennai Super Kings      5.1875
8   2008.0      91.0  Chennai Super Kings      5.1875
9   2008.0      96.0  Chennai Super Kings      5.1875
10  2008.0     103.0  Chennai Super Kings      5.1875
11  2008.0     107.0  Chennai Super Kings      5.1875
12  2008.0     110.0  Chennai Super Kings      5.1875
13  2008.0     114.0  Chennai Super Kings      5.1875
14  2008.0     116.0  Chennai Super Kings      5.1875
15  2008.0     117.0  Chennai Super Kings      5.1875
    season  match_id     bowling_team  avgWickets
0   2008.0      64.0  Deccan Cha

    season  match_id      bowling_team  avgWickets
0   2009.0     120.0  Delhi Daredevils    6.066667
1   2009.0     125.0  Delhi Daredevils    6.066667
2   2009.0     129.0  Delhi Daredevils    6.066667
3   2009.0     133.0  Delhi Daredevils    6.066667
4   2009.0     136.0  Delhi Daredevils    6.066667
5   2009.0     141.0  Delhi Daredevils    6.066667
6   2009.0     146.0  Delhi Daredevils    6.066667
7   2009.0     150.0  Delhi Daredevils    6.066667
8   2009.0     154.0  Delhi Daredevils    6.066667
9   2009.0     158.0  Delhi Daredevils    6.066667
10  2009.0     161.0  Delhi Daredevils    6.066667
11  2009.0     165.0  Delhi Daredevils    6.066667
12  2009.0     167.0  Delhi Daredevils    6.066667
13  2009.0     170.0  Delhi Daredevils    6.066667
14  2009.0     172.0  Delhi Daredevils    6.066667
    season  match_id     bowling_team  avgWickets
0   2009.0     120.0  Kings XI Punjab    5.071429
1   2009.0     123.0  Kings XI Punjab    5.071429
2   2009.0     127.0  Kings XI Pun

    season  match_id    bowling_team  avgWickets
0   2010.0     176.0  Mumbai Indians       5.375
1   2010.0     183.0  Mumbai Indians       5.375
2   2010.0     188.0  Mumbai Indians       5.375
3   2010.0     191.0  Mumbai Indians       5.375
4   2010.0     194.0  Mumbai Indians       5.375
5   2010.0     199.0  Mumbai Indians       5.375
6   2010.0     201.0  Mumbai Indians       5.375
7   2010.0     207.0  Mumbai Indians       5.375
8   2010.0     211.0  Mumbai Indians       5.375
9   2010.0     215.0  Mumbai Indians       5.375
10  2010.0     219.0  Mumbai Indians       5.375
11  2010.0     221.0  Mumbai Indians       5.375
12  2010.0     226.0  Mumbai Indians       5.375
13  2010.0     230.0  Mumbai Indians       5.375
14  2010.0     231.0  Mumbai Indians       5.375
15  2010.0     234.0  Mumbai Indians       5.375
    season  match_id      bowling_team  avgWickets
0   2010.0     176.0  Rajasthan Royals    4.714286
1   2010.0     180.0  Rajasthan Royals    4.714286
2   2010.0    

    season  match_id   bowling_team  avgWickets
0   2011.0     239.0  Pune Warriors    5.142857
1   2011.0     244.0  Pune Warriors    5.142857
2   2011.0     250.0  Pune Warriors    5.142857
3   2011.0     254.0  Pune Warriors    5.142857
4   2011.0     262.0  Pune Warriors    5.142857
5   2011.0     264.0  Pune Warriors    5.142857
6   2011.0     268.0  Pune Warriors    5.142857
7   2011.0     271.0  Pune Warriors    5.142857
8   2011.0     277.0  Pune Warriors    5.142857
9   2011.0     284.0  Pune Warriors    5.142857
10  2011.0     286.0  Pune Warriors    5.142857
11  2011.0     295.0  Pune Warriors    5.142857
12  2011.0     298.0  Pune Warriors    5.142857
13  2011.0     301.0  Pune Warriors    5.142857
    season  match_id      bowling_team  avgWickets
0   2011.0     236.0  Rajasthan Royals    4.230769
1   2011.0     241.0  Rajasthan Royals    4.230769
2   2011.0     246.0  Rajasthan Royals    4.230769
3   2011.0     251.0  Rajasthan Royals    4.230769
4   2011.0     256.0  Raj

    season  match_id      bowling_team  avgWickets
0   2012.0     311.0  Rajasthan Royals         5.5
1   2012.0     314.0  Rajasthan Royals         5.5
2   2012.0     319.0  Rajasthan Royals         5.5
3   2012.0     322.0  Rajasthan Royals         5.5
4   2012.0     326.0  Rajasthan Royals         5.5
5   2012.0     328.0  Rajasthan Royals         5.5
6   2012.0     334.0  Rajasthan Royals         5.5
7   2012.0     338.0  Rajasthan Royals         5.5
8   2012.0     345.0  Rajasthan Royals         5.5
9   2012.0     349.0  Rajasthan Royals         5.5
10  2012.0     354.0  Rajasthan Royals         5.5
11  2012.0     358.0  Rajasthan Royals         5.5
12  2012.0     361.0  Rajasthan Royals         5.5
13  2012.0     365.0  Rajasthan Royals         5.5
14  2012.0     373.0  Rajasthan Royals         5.5
15  2012.0     377.0  Rajasthan Royals         5.5
    season  match_id                 bowling_team  avgWickets
0   2012.0     312.0  Royal Challengers Bangalore         5.4
1   2012.

    season  match_id                 bowling_team  avgWickets
0   2013.0     383.0  Royal Challengers Bangalore         6.0
1   2013.0     388.0  Royal Challengers Bangalore         6.0
2   2013.0     392.0  Royal Challengers Bangalore         6.0
3   2013.0     396.0  Royal Challengers Bangalore         6.0
4   2013.0     401.0  Royal Challengers Bangalore         6.0
5   2013.0     407.0  Royal Challengers Bangalore         6.0
6   2013.0     411.0  Royal Challengers Bangalore         6.0
7   2013.0     417.0  Royal Challengers Bangalore         6.0
8   2013.0     420.0  Royal Challengers Bangalore         6.0
9   2013.0     426.0  Royal Challengers Bangalore         6.0
10  2013.0     429.0  Royal Challengers Bangalore         6.0
11  2013.0     432.0  Royal Challengers Bangalore         6.0
12  2013.0     438.0  Royal Challengers Bangalore         6.0
13  2013.0     441.0  Royal Challengers Bangalore         6.0
14  2013.0     448.0  Royal Challengers Bangalore         6.0
15  2013

    season  match_id         bowling_team  avgWickets
0   2015.0     519.0  Chennai Super Kings    6.117647
1   2015.0     521.0  Chennai Super Kings    6.117647
2   2015.0     527.0  Chennai Super Kings    6.117647
3   2015.0     530.0  Chennai Super Kings    6.117647
4   2015.0     533.0  Chennai Super Kings    6.117647
5   2015.0     538.0  Chennai Super Kings    6.117647
6   2015.0     542.0  Chennai Super Kings    6.117647
7   2015.0     547.0  Chennai Super Kings    6.117647
8   2015.0     551.0  Chennai Super Kings    6.117647
9   2015.0     554.0  Chennai Super Kings    6.117647
10  2015.0     559.0  Chennai Super Kings    6.117647
11  2015.0     563.0  Chennai Super Kings    6.117647
12  2015.0     565.0  Chennai Super Kings    6.117647
13  2015.0     569.0  Chennai Super Kings    6.117647
14  2015.0     573.0  Chennai Super Kings    6.117647
15  2015.0     575.0  Chennai Super Kings    6.117647
16  2015.0     576.0  Chennai Super Kings    6.117647
    season  match_id      bo

    season  match_id     bowling_team  avgWickets
0   2016.0     579.0  Kings XI Punjab         4.5
1   2016.0     583.0  Kings XI Punjab         4.5
2   2016.0     586.0  Kings XI Punjab         4.5
3   2016.0     589.0  Kings XI Punjab         4.5
4   2016.0     594.0  Kings XI Punjab         4.5
5   2016.0     597.0  Kings XI Punjab         4.5
6   2016.0     604.0  Kings XI Punjab         4.5
7   2016.0     608.0  Kings XI Punjab         4.5
8   2016.0     612.0  Kings XI Punjab         4.5
9   2016.0     615.0  Kings XI Punjab         4.5
10  2016.0     619.0  Kings XI Punjab         4.5
11  2016.0     622.0  Kings XI Punjab         4.5
12  2016.0     626.0  Kings XI Punjab         4.5
13  2016.0     629.0  Kings XI Punjab         4.5
    season  match_id           bowling_team  avgWickets
0   2016.0     578.0  Kolkata Knight Riders         5.4
1   2016.0     581.0  Kolkata Knight Riders         5.4
2   2016.0     584.0  Kolkata Knight Riders         5.4
3   2016.0     589.0  Kolk

    season  match_id    bowling_team  avgWickets
0   2017.0       2.0  Mumbai Indians    5.823529
1   2017.0       7.0  Mumbai Indians    5.823529
2   2017.0      10.0  Mumbai Indians    5.823529
3   2017.0      12.0  Mumbai Indians    5.823529
4   2017.0      16.0  Mumbai Indians    5.823529
5   2017.0      22.0  Mumbai Indians    5.823529
6   2017.0      24.0  Mumbai Indians    5.823529
7   2017.0      28.0  Mumbai Indians    5.823529
8   2017.0      34.0  Mumbai Indians    5.823529
9   2017.0      37.0  Mumbai Indians    5.823529
10  2017.0      44.0  Mumbai Indians    5.823529
11  2017.0      47.0  Mumbai Indians    5.823529
12  2017.0      50.0  Mumbai Indians    5.823529
13  2017.0      53.0  Mumbai Indians    5.823529
14  2017.0      56.0  Mumbai Indians    5.823529
15  2017.0      58.0  Mumbai Indians    5.823529
16  2017.0      59.0  Mumbai Indians    5.823529
    season  match_id            bowling_team  avgWickets
0   2017.0       2.0  Rising Pune Supergiant        6.25
1   

    season  match_id                 bowling_team  avgWickets
0   2018.0    7896.0  Royal Challengers Bangalore    5.285714
1   2018.0    7901.0  Royal Challengers Bangalore    5.285714
2   2018.0    7904.0  Royal Challengers Bangalore    5.285714
3   2018.0    7907.0  Royal Challengers Bangalore    5.285714
4   2018.0    7912.0  Royal Challengers Bangalore    5.285714
5   2018.0    7917.0  Royal Challengers Bangalore    5.285714
6   2018.0    7922.0  Royal Challengers Bangalore    5.285714
7   2018.0    7924.0  Royal Challengers Bangalore    5.285714
8   2018.0    7928.0  Royal Challengers Bangalore    5.285714
9   2018.0    7932.0  Royal Challengers Bangalore    5.285714
10  2018.0    7938.0  Royal Challengers Bangalore    5.285714
11  2018.0    7941.0  Royal Challengers Bangalore    5.285714
12  2018.0    7944.0  Royal Challengers Bangalore    5.285714
13  2018.0    7946.0  Royal Challengers Bangalore    5.285714
    season  match_id         bowling_team  avgWickets
0   2018.0    78

In [175]:
team_wickets

,season,match_id,bowling_team,avgWickets
0,2008,60,Kolkata Knight Riders,4.0
1,2008,60,Royal Challengers Bangalore,4.0
2,2008,61,Chennai Super Kings,5.0
3,2008,61,Kings XI Punjab,5.0
4,2008,62,Delhi Daredevils,5.0
...,...,...,...,...
1505,2019,11413,Sunrisers Hyderabad,5.0
1506,2019,11414,Chennai Super Kings,6.0
1507,2019,11414,Delhi Capitals,6.0
1508,2019,11415,Chennai Super Kings,6.0


In [170]:
dict2

{(2008, 'Chennai Super Kings'): 5.1875,
 (2008, 'Deccan Chargers'): 4.285714285714286,
 (2008, 'Delhi Daredevils'): 5.857142857142857,
 (2008, 'Kings XI Punjab'): 5.533333333333333,
 (2008, 'Kolkata Knight Riders'): 4.6923076923076925,
 (2008, 'Mumbai Indians'): 5.928571428571429,
 (2008, 'Rajasthan Royals'): 6.0,
 (2008, 'Royal Challengers Bangalore'): 4.0,
 (2009, 'Chennai Super Kings'): 6.0,
 (2009, 'Deccan Chargers'): 6.0625,
 (2009, 'Delhi Daredevils'): 6.066666666666666,
 (2009, 'Kings XI Punjab'): 5.071428571428571,
 (2009, 'Kolkata Knight Riders'): 3.769230769230769,
 (2009, 'Mumbai Indians'): 5.538461538461538,
 (2009, 'Rajasthan Royals'): 6.076923076923077,
 (2009, 'Royal Challengers Bangalore'): 4.9375,
 (2010, 'Chennai Super Kings'): 5.625,
 (2010, 'Deccan Chargers'): 5.8125,
 (2010, 'Delhi Daredevils'): 5.785714285714286,
 (2010, 'Kings XI Punjab'): 4.642857142857143,
 (2010, 'Kolkata Knight Riders'): 4.857142857142857,
 (2010, 'Mumbai Indians'): 5.375,
 (2010, 'Rajasthan 